In [2]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
from itertools import product, cycle
from heapq import heapify, heappush, heappop
import cv2
import omnifig as fig
import numpy as np
np.set_printoptions(linewidth=120)
import pickle
from tabulate import tabulate
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
# import torchvision.models
from torch.utils.data import Dataset, DataLoader, TensorDataset
import networkx as nx

%matplotlib notebook
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import animation
import matplotlib as mpl
from matplotlib.collections import PatchCollection
# mpl.rc('image', cmap='gray')
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize

fig.initialize()

import omnilearn as learn
from omnilearn import models
from omnilearn import util
from omnilearn.data import InterventionSamplerBase
from omnilearn import viz as viz_util

from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA

from src import sample_full_interventions, response_mat, factor_reponses
from src.responses import sample_full_interventions

# import gpumap
from c3linearize import linearize, class_graph


In [3]:
device = 'cuda'
# device = 'cuda:1'
dataset = None
root = Path(r'C:/Users/anwan/Documents/workspace/rfd_models/')
figure_root = Path('figures/responses')
figure_root = None
if figure_root is not None and not figure_root.exists():
    figure_root.mkdir()
seed = 10
B = 128
N = 16
G = 32

In [4]:
run_names = '''rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64_210524-161408
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-wdp0001_210524-161607
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527'''.split('\n')
print(tabulate(enumerate(run_names)))

-  ---------------------------------------------------------------------------
0  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64_210524-161408
1  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-wdp0001_210524-161607
2  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
3  rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
4  rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
5  rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
6  rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
7  rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009
8  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527
-  ---------------------------------------------------------------------------


In [7]:
print('\n'.join([f'unbuffer fig eval-fix-hybrid eval/rfd-fid --load {r} --eval.mode test --eval.ident final' for r in run_names]))

unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64_210524-161408 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-wdp0001_210524-161607 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633 --eval.mode test --eval.ident final
unbuffer fig eval-fix-hybrid eval/rfd-fid --load rfd_vae1_e-l16_d-l16_d16-strd-tra

In [4]:
run_name = random.choice(run_names)

run_name = run_names[2]
run_name = run_names[3]

run_name

'rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908'

In [5]:
load_config = fig.get_config(path=run_name, root=str(root), **{'override.device':device})
load_config.set_silent(True)
run = fig.run('load-run', load_config)
A = run.get_config()
A.push('model-override.model._model_mod.hybrid', 1)
A.set_silent(True)
info = A.pull('info')
info

ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"


{'arch_type': 'c16',
 'dataset_type': 'rfd',
 'dec_type': 'c16',
 'enc_type': 'c16',
 'extra': 'd16-strd-trans-lrp00005-bs64-ch256',
 'model_type': 'ae',
 'size': 128}

In [6]:
model = run.get_model()
model.switch_to('val')
model.to(device);

Hybrid_Autoencoder(
  (encoder): MultiLayer(
    (layers): ModuleList(
      (0): ConvLayer(
        residual=False
        (conv): Conv2d(3, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (1): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (2): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (3): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): 

Loaded parameters from C:\Users\anwan\Documents\workspace\rfd_models\rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908\ckpt200000


In [7]:
# data = run.get_results('eval')
# data.keys()

In [8]:
# data['hybrid_fid']

In [9]:
if dataset is None:
    dataset = run.get_dataset()
len(dataset)

====== Opening RFD Dataset ======


720000

In [10]:
dataset.get_available_modes()

{'heldout_test', 'real_test', 'test', 'train'}

In [11]:
batch = dataset.get_batch(batch_size=128, shuffle=True)
batch.shape

torch.Size([128, 3, 128, 128])

In [12]:
with torch.no_grad():
    Q = model.encode(batch)
    if isinstance(Q, distrib.Normal):
        Q = Q.loc
    if model._latent is None:
        model._latent = Q
Q.shape

torch.Size([128, 16])

In [13]:
A.update({
    'novel': True,
    'skip-expensive': True,
    'dataset.batch_size': 16,
    'evaluation': None,
    'inline': True,
    'pbar._type': 'progress-bar',
    'pbar.display-on': 'jupyter',
    'fid.dim': 2048,
    'fid.n_samples': 50000,
    'fid.batch_size': 32,
})

In [ ]:
out = run.evaluate(overwrite=True)

Will save results to C:\Users\anwan\Documents\workspace\rfd_models\rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
Loading inception model dim=2048


  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
out.keys()

In [25]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 16.907577234205405,
 'prior_fid': 35.69107753449731,
 'hybrid_fid': 28.269544532424334}

In [25]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 16.907577234205405,
 'prior_fid': 35.69107753449731,
 'hybrid_fid': 28.269544532424334}

In [15]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 9.521835270454687, 'hybrid_fid': 19.730062040454442}